# Artificial Neural Networks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError, RootMeanSquaredError
from sklearn.model_selection import KFold
from numpy import sqrt
import tensorflow as tf
tf.config.optimizer.set_jit(True)
import pytz
import datetime
import time
import os
import joblib

In [4]:
df = pd.read_csv('../data/df_pkill.csv', delimiter = ',',header=0)
df

,alt_sht,vel_sht,pit_sht,alt_tgt,vel_tgt,hdg_tgt,rgt_tgt,dist,delay,turn_dg,pkill
0,1900.094833,516.257515,-9.115002,9411.674491,389.059516,-179.508724,28.173016,22.107204,24.962478,5.107690,0.087
1,1668.030968,384.114554,-9.291737,6424.170046,538.332386,150.564222,21.635788,42.957053,27.889110,140.635249,0.016
2,1430.046650,552.946387,-16.802029,11292.969864,536.495037,-71.258760,47.099787,17.904767,28.814680,13.967480,0.070
3,1819.812543,524.681447,2.326836,3983.583185,354.232941,-147.655375,-25.450868,38.858097,25.736859,47.110727,0.020
4,1348.600786,367.370819,13.801087,8269.417066,533.015720,85.274320,-47.226769,27.176054,17.912385,137.619306,0.024
...,...,...,...,...,...,...,...,...,...,...,...
2855050,44376.436551,544.473934,-4.773440,16868.508237,620.848721,83.157352,32.501000,32.909110,21.978916,71.510008,0.031
2855051,44396.062176,555.458863,-12.651975,44331.857296,626.047542,28.146378,-26.005787,42.962424,27.934929,145.538436,0.024
2855052,44373.716110,609.965112,-24.517640,16754.178594,580.015564,168.638639,-16.550910,43.350364,15.081863,79.534893,0.043
2855053,44432.347726,617.846742,11.024131,24860.378262,658.322238,-145.516952,14.465779,40.760772,26.462078,84.072836,0.111


### Train Test Split

In [3]:
samples_per_bin, bins = np.histogram(df['pkill'], bins='doane') # Doane's method
df['bins'] = pd.DataFrame(np.digitize(df['pkill'], bins))

X = df.drop(['pkill','bins'],axis=1)
y = df[['pkill']]
b = df['bins']

#X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42, stratify=b)

### Scaling

In [4]:
np.histogram(df['pkill'], bins='doane') 

(array([2356081,   11604,    3947,    2182,    1320,     961,     712,
            541,     449,     351,     316,     303,     227,     219,
            198,     169,     175,     168,     222,     317,     413,
          13348,   62782,   52349,   45148,   39765,   35620,   32563,
          30665,   30623,   32014,   37159,   62144]),
 array([1.20000000e-02, 3.00063636e+00, 5.98927273e+00, 8.97790909e+00,
        1.19665455e+01, 1.49551818e+01, 1.79438182e+01, 2.09324545e+01,
        2.39210909e+01, 2.69097273e+01, 2.98983636e+01, 3.28870000e+01,
        3.58756364e+01, 3.88642727e+01, 4.18529091e+01, 4.48415455e+01,
        4.78301818e+01, 5.08188182e+01, 5.38074545e+01, 5.67960909e+01,
        5.97847273e+01, 6.27733636e+01, 6.57620000e+01, 6.87506364e+01,
        7.17392727e+01, 7.47279091e+01, 7.77165455e+01, 8.07051818e+01,
        8.36938182e+01, 8.66824545e+01, 8.96710909e+01, 9.26597273e+01,
        9.56483636e+01, 9.86370000e+01]))

In [94]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.values
y_test = y_test.values

if not os.path.exists('./models/scaler'):
    os.makedirs('./models/scaler')
joblib.dump(scaler, './models/scaler/scaler.pkl');

### Hyperparameters

In [95]:
patience = 10
epochs = 1_000_000
batch_size = 32
n_splits = 5
p = 0.20
list_of_hidden_layers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
list_of_units = [16, 32, 64, 128, 256]

### Training process

In [ ]:
for hidden_layers in list_of_hidden_layers:
    for units in list_of_units:
        
        print(f'Starting a new training process! with {hidden_layers} hidden layers with {units} units each')
        
        metrics = pd.DataFrame(columns=['fold', 'mae', 'mse', 'rmse', 'r2', 'training_time', 'inference_time'])

        # Define the K-fold Cross Validator
        kfold = KFold(n_splits=n_splits, shuffle=True)

        # K-fold Cross Validation model evaluation
        fold_no = 1
        for train, val in kfold.split(X_train, y_train):

            # Define the model architecture
            model = Sequential()

            #input layer
            model.add(Dense(units=X.shape[1], activation='relu'))
            #https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/#:~:text=Dropout%20Rate,-The%20default%20interpretation&text=A%20good%20value%20for%20dropout,rate%2C%20such%20as%20of%200.8.
            #model.add(Dropout(p))

            #hidden layers

            for i in range(hidden_layers):
                model.add(Dense(units=units,activation='relu'))
                #model.add(Dropout(p))

            #output layer
            model.add(Dense(units=1))

            early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)

            # Compile the model for a regressionn problem
            model.compile(loss='mse', optimizer='adam', metrics=[MeanAbsoluteError(),MeanSquaredError(),RootMeanSquaredError()])

            # Generate a print
            print('------------------------------------------------------------------------')
            print(f'Training for fold {fold_no} ...')

            # get the start time
            st_wall = time.time()

            # Fit data to model
            history = model.fit(X_train[train], y_train[train],
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(X_train[val], y_train[val]), 
                  callbacks=[early_stop])

            # get the end time
            et_wall = time.time()

            # get execution time
            wall_time = et_wall - st_wall

            print('Training Execution time:', wall_time, 'seconds')

            # get the start time
            st_wall_inf = time.time()

            # Generate generalization metrics
            y_pred = model.predict(X_test)

            # get the end time
            et_wall_inf = time.time()

            # get execution time
            wall_time_inf = et_wall_inf - st_wall_inf

            print('Inference Execution time:', wall_time_inf, 'seconds')
            print(r2_score(y_test, y_pred))

            scores = [fold_no, mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), 
                      sqrt(mean_squared_error(y_test, y_pred)), r2_score(y_test, y_pred), wall_time, wall_time_inf]

            metrics.loc[len(metrics)] = scores

            # Increase fold number
            fold_no = fold_no + 1


        metrics.loc[len(metrics)] = ['mean', metrics['mae'].mean(), metrics['mse'].mean(), metrics['rmse'].mean(), metrics['r2'].mean(), metrics['training_time'].mean(), metrics['inference_time'].mean()]
        metrics.loc[len(metrics)] = ['std', metrics['mae'].iloc[:-1].std(), metrics['mse'].iloc[:-1].std(), metrics['rmse'].iloc[:-1].std(), metrics['r2'].iloc[:-1].std(), metrics['training_time'].iloc[:-1].std(), metrics['inference_time'].iloc[:-1].std()]  
        metrics.loc[len(metrics)] = ['sum', metrics['mae'].iloc[:-2].sum(), metrics['mse'].iloc[:-2].sum(), metrics['rmse'].iloc[:-2].sum(), metrics['r2'].iloc[:-2].sum(), metrics['training_time'].iloc[:-2].sum(), metrics['inference_time'].iloc[:-2].sum()]
        metrics = metrics.set_index('fold')
        
        
        path_to_save = f'./results/{hidden_layers}_hidden_layers/'

        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)

        metrics.to_csv(f'{path_to_save}metrics_{units}_units.csv')
        
        
        path_to_save = f'./models/{hidden_layers}_hidden_layers/'

        model.save(f'{path_to_save}model_{units}_units.h5')  

Starting a new training process! with 1 hidden layers with 16 units each
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/1000000
57102/57102 [==============================] - 54s 946us/step - loss: 392.5019 - mean_absolute_error: 11.3170 - mean_squared_error: 392.5019 - root_mean_squared_error: 19.6559 - val_loss: 265.9884 - val_mean_absolute_error: 8.5797 - val_mean_squared_error: 265.9884 - val_root_mean_squared_error: 16.3092
Epoch 2/1000000
57102/57102 [==============================] - 54s 948us/step - loss: 257.9463 - mean_absolute_error: 8.2960 - mean_squared_error: 257.9463 - root_mean_squared_error: 16.0600 - val_loss: 236.7663 - val_mean_absolute_error: 7.6939 - val_mean_squared_error: 236.7663 - val_root_mean_squared_error: 15.3872
Epoch 3/1000000
57102/57102 [==============================] - 55s 965us/step - loss: 233.5010 - mean_absolute_error: 7.4396 - mean_squared_error: 233.5010 - root_mean_squared_error: 15.280